In [ ]:
import os
import sys
import random

import numpy as np
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

seed = 2019
random.seed = seed
np.random.seed = seed
tf.seed = seed

In [ ]:

class DataGen(keras.utils.Sequence):
    def __init__(self, ids, path, batch_size=15, image_size=512, shuffle = True):
        self.ids = ids
        self.path = path
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __load__(self, id_name):
        ## Path
        image_path = os.path.join(self.path, id_name, id_name + '_image', id_name) + ".png"
        mask_path = os.path.join(self.path, id_name, id_name + '_mask', id_name) + ".png"
        #all_masks = os.listdir(mask_path)
        
        ## Reading Image
        image = cv2.imread(image_path, 1)
        image = cv2.resize(image, (self.image_size, self.image_size))
        
        mask = np.zeros((self.image_size, self.image_size, 1))
        
        ## Reading Masks
        mask_path = mask_path
        _mask_image = cv2.imread(mask_path, -1)
        _mask_image = cv2.resize(_mask_image, (self.image_size, self.image_size)) #512x512
        _mask_image = np.expand_dims(_mask_image, axis=-1)
        mask = np.maximum(mask, _mask_image)
        
        ## Normalizaing 
        image = image/255.0
        mask = mask/255.0
        
        return image, mask
    
    def __getitem__(self, index):
        if(index+1)*self.batch_size > len(self.ids):
            self.batch_size = len(self.ids) - index*self.batch_size
        
        files_batch = self.ids[index*self.batch_size : (index+1)*self.batch_size]
        
        image = []
        mask  = []
        for id_name in files_batch:
            _img, _mask = self.__load__(id_name)
            image.append(_img)
            mask.append(_mask)
            
        image = np.array(image)
        mask  = np.array(mask)
        
        return image, mask
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.ids))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __len__(self):
        return int(np.ceil(len(self.ids)/float(self.batch_size)))


In [ ]:
image_size = 512
train_path = r"C:/Users/pyash/Desktop/dataset/train/"
test_path = r"C:/Users/pyash/Desktop/dataset/test/"
epochs = 9
batch_size = 15

## Training Ids
train_ids = next(os.walk(train_path))[1]
test_ids = next(os.walk(test_path))[1]

valid_ids = test_ids[0:51]
#test_ids = test_ids[20:]

print(valid_ids)
len(valid_ids)

In [ ]:
gen = DataGen(test_ids, test_path, batch_size=batch_size, image_size=image_size)
x, y = gen.__getitem__(0)
print(x.shape, y.shape)

In [ ]:
r = random.randint(0, len(x)-1)
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)
ax = fig.add_subplot(1, 2, 1)
ax.imshow(x[r])
ax = fig.add_subplot(1, 2, 2)
ax.imshow(np.reshape(y[r], (image_size, image_size)), cmap="gray")

In [ ]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

In [ ]:
def UNet():
    f = [16, 32, 64, 128, 256, 512]
    inputs = keras.layers.Input((image_size, image_size, 3))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #256--->128
    c2, p2 = down_block(p1, f[1]) #128--->64
    c3, p3 = down_block(p2, f[2]) #64--->32
    c4, p4 = down_block(p3, f[3]) #32--->16
    c5, p5 = down_block(p4, f[4]) #16--->8
    
    bn = bottleneck(p5, f[5])
    
    u1 = up_block(bn, c5, f[4]) #8--->16
    u2 = up_block(u1, c4, f[3]) #16--->32
    u3 = up_block(u2, c3, f[2]) #32--->64
    u4 = up_block(u3, c2, f[1]) #64--->128
    u5 = up_block(u4, c1, f[0]) #128--->256
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u5)
    model = keras.models.Model(inputs, outputs)
    return model

In [ ]:
model = UNet()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])
model.summary()

In [ ]:
train_steps = len(train_ids)//batch_size
valid_steps = len(valid_ids)//batch_size

model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=train_steps, validation_steps=valid_steps, epochs=epochs)

In [ ]:
## Save the Weights
#model.save_weights("UNetW.h5")
model.load_weights('UnetW.h5')

train_gen = DataGen(train_ids, train_path, image_size=image_size, batch_size=batch_size)
valid_gen = DataGen(valid_ids, test_path, image_size=image_size, batch_size=batch_size)

## Dataset for prediction
x, y = valid_gen.__getitem__(0)
print (x.shape, y.shape)
result = model.predict(x)

result = result > 0.5


In [ ]:
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)
r=1

ax = fig.add_subplot(1, 2, 1)
ax.imshow(np.reshape(y[r]*255, (image_size, image_size)), cmap="gray")

ax = fig.add_subplot(1, 2, 2)
ax.imshow(np.reshape(result[r]*255, (image_size, image_size)), cmap="gray")

In [ ]:
iou_score_sum = 0
for r in range(0,15):
    intersection = np.logical_and(y[r], result[r])
    union = np.logical_or(y[r], result[r])
    iou_score = np.sum(intersection) / np.sum(union)
    print(str(r) + '. IOU score: ' + str(iou_score))
    iou_score_sum = iou_score_sum + iou_score
    
iou_score_mean = iou_score_sum/len(y)
print('\nMean IOU score: ' + str(iou_score_mean))